# PostgreSQLLoader
> **PostgreSQLLoader**:
PostgreSQLLoader lets you load data from Cloud SQL for PostgreSQL database as Langchain Documents. This tool also provides a PostgreSQLDocumentSaver Class to save Langchain Documents back into Cloud SQL for PostgreSQL database

This tutorial illustrates how to work with an end-to-end data Document Loader and Document Saver system in LangChain for CloudSQL for PostgreSQL.

## Pre-requisites

### Install the library

In [ ]:
! pip install langchain langchain-community google-cloud asyncio asyncpg langchain_google_cloud_sql_pg
! pip install "cloud-sql-python-connector[asyncpg]"

:**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Note

`If you do not have a GCP project, please follow the below link to create a new project`

[Create a Google Cloud project](https://developers.google.com/workspace/guides/create-project)


#### Set your project ID

If you don't know your project ID, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113).

In [ ]:
# @title Project { display-mode: "form" }
PROJECT_ID = "gcp_project_id"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Set the region

You can also change the `REGION` variable used by CloudSQL Postgres. Learn more about [CloudSQL Postgres regions](https://cloud.google.com/sql/docs/postgres/locations).

In [ ]:
# @title Region { display-mode: "form" }
REGION = "US"  # @param {type: "string"}

#### Set the dataset and table names

They will be your CloudSQL Postgres Document Table

In [ ]:
# @title Instance,  Database and Table { display-mode: "form" }
INSTANCE = "my_cloudsql_instance" # @param {type: "string"}
DATABASE = "my_langchain_database"  # @param {type: "string"}
TABLE = "doc_table"  # @param {type: "string"}

### Pre-requisites for connecting to the CloudSQL instance

To connect to the postgreSQL instance make sure to setup the cloudSQL auth proxy and ensure the addition of IAM users to the list of authenticated users to connect to the instance.

Refer to this [link](https://github.com/GoogleCloudPlatform/cloud-sql-proxy) to setup auth proxy.

Refer to this [link](https://cloud.google.com/sql/docs/postgres/users?_ga=2.165429503.-1722697531.1694071937) to add users to the instance

### Authenticating your notebook environment

- If you are using **Colab** to run this notebook, uncomment the cell below and continue.
- If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
from google.colab import auth as google_auth

google_auth.authenticate_user()

## PostgreSQLLoader Class Sample
PostgreSQL Loader Class

### Import PostgreSQL Loader Class [instance](https://)

In [ ]:
from langchain_google_cloud_sql_pg import PostgreSQLLoader, PostgreSQLDocumentSaver, PostgreSQLEngine

### Create PostgreSQLEngine class to connect to the database

In [ ]:
# PostgreSQLLoader requires an engine created using the CloudSQLEngine class
engine = PostgreSQLEngine.from_instance(
    region = "region_name",
    instance = "instance_name",
    database = "dbname"
)

### Create PostgreSQLLoader to create a table

In [ ]:
# Creating a basic PostgreSQLLoader object
loader = PostgreSQLLoader(
    engine,
    table_name=table_name,
    query=query,
    content_columns=content_columns,
    metadata_columns=metadata_columns,
    format=format,
    read_only=read_only,
    time_out=time_out,
    formatter=formatter,
)

# Parameters:
# engine - a PostgreSQL Engine object used to connect to the database
# table_name - (Optional) the table name (OneOf: table_name, query)
# query - (Optional) a SQL query (OneOf: table_name, query)
# content_columns - list of column(s) or field(s) to use for a Document’s pagecontent. Page content is the default field for embeddings generation. (Default:“page_content”)
# metadata_columns - list of column(s) or field(s) to use for metadata. (Default:“langchain_metadata”)
# format - set the format of page content if using multiple columns or fields. (Default:“text” space separated string concatenation) Other format: JSON, YAML, CSV, etc.
# read_only - Prevent queries that write to the database (default: True)
# timeout - specify a timeout for the client in seconds
# fomatter - a custom formatter object

### Load Documents via default table
The loader returns a list of Documents from the table using the first column as page_content and all other columns as metadata. The default table will have the first column as
page_content and the second column as metadata (JSON). Each row becomes a document.

In [ ]:
loader = PostgreSQLLoader(
     engine,
     table_name="my-docs-table" 
   )
docs = loader.load()
# or lazy load in async contenct
docs = loader.alazy_load()
## [Document(page_content="...", metadata={...}), ...]

### Load documents via custom table/metadata or custom page content columns
The loader returns a list of Documents from the table using the first column as page_content and all other columns as metadata. 
If content_columns are provided, then all colums specified will be loaded into page_content separated by " " and all other columns as metadata
If a JSON column is listed as a metadata column with the name, “langchain_metadata”, it will be used as the base dictionary. Other column data will be added and may overwrite the original value. Each row becomes a document.

In [ ]:
loader = PostgreSQLLoader(
    engine,
    table_name="my-docs-table"
    content_columns=["product_name"] # Optional
    metadata_columns=["id"] # Optional
   )
docs = loader.alazy_load()
   ## [Document(page_content="...", metadata={...}), ...]

### Set page content format

The loader returns a list of Documents, one document per row, where page content defaults to the first columns present in the query and metadata defaults to all other columns. Use with content_columns to overwrite the column used for page content. Use metadata_columns to select specific metadata columns rather than using all remaining columns.
If content_columns are provided, then all colums specified will be loaded into page_content separated by " " and all other columns as metadata
If a JSON column is listed as a metadata column with the name, “langchain_metadata”, it will be used as the base dictionary. Other column data will be added and may overwrite the original value. Each row becomes a document.

In [ ]:
# Used with "SELECT *" query
loader = PostgreSQLLoader(
     engine,
     query="SELECT * FROM products",
   )
docs = loader.alazy_load()
## Example doc: Document(page_content="123",
metadata={"product_name":"cards",price=10,"category":"toys"})
# Used with specific columns
loader = PostgreSQLLoader(
     engine,
     query="SELECT product_name, description FROM products",
)
docs = loader.alazy_load()
## Example doc: Document(page_content="cards", metadata={"description": "playing cards are cool!"})
# Used with content_columns argument

### Set page content format
The loader returns a list of Documents, with one document per row, with page content in speci ed string format, i.e. text (space separated concatenation), JSON, YAML, CSV, etc. JSON and YAML formats include headers, while text and CSV do not include field headers.


In [ ]:
loader = PostgreSQLLoader(
     engine,
     table_name="products",
     content_columns=["product_name", "description"],
     format="YAML"
)
## Example doc: Document(page_content="product_name: cards\ndescription: playing cards are cool!", metadata={})


### Use custom page content formatter
Instead of using one of the built-in page content formats, the user can provide their own function for generating a custom string format.

In [ ]:
# Define a custom format function
   def my_formatter(**kwargs):
          return '-'.join(kwargs.values())
       
loader = PostgreSQLLoader(
     engine,
     table_name="products",
     content_columns=["product_name", "description"],
     formatter=my_formatter
)
## Example doc: Document(page_content="cards-playing cards are cool!", metadata={})


### Set timeout for query and Read only query protection
Using the table_name field over the query field will automatically get read only protection. This defaults to True.
Set timeout for the loader. This may be helpful for large datasets.


In [ ]:
loader = PostgreSQLLoader(
     engine,
     query="SELECT * FROM products",
     timeout=30s,
     read_only=False, # Use to change the default to False
   )

## PostgreSQLDocumentSaver Class Sample
This class allows for saving of pre-processed documents. This table can easily be loaded via a Document Loader or updated to be a VectorStore.

In [3]:
### Follow the PostgreSQL Engine creation as in the above example

### Add Documents
Save documents in the DocumentSaver table. See Defaults for the table schema. Document’s metadata is added to columns if found or stored in langchain_metadata JSON column.

In [ ]:
saver = PostgreSQLDocumentSaver(engine, table_name)
saver.aadd_documents(docs)

### Delete Documents
Delete all instances of a document from the DocumentSaver table by matching the entire Document object.

In [ ]:
# Load the doc
loader = PostgreSQLDocumentSaver(engine, "SELECT * FROM table_name WHERE page_content =='cards'")
docs = loader.alazy_load()
# Init saver and delete
saver = PostgreSQLDocumentSaver(engine, table_name)
saver.adelete(docs)